In [347]:
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow_datasets as tfds
from keras.datasets import mnist
import numpy as np
import pandas as pd
import tensorflow as tf

In [348]:
train = pd.read_csv("train.csv")
y_final = train["label"]
train = train.drop("label", axis=1)
train = train.values.reshape(42000,28,28,1)
test = pd.read_csv("test.csv")

In [349]:
MNIST = tfds.image.MNIST()

In [350]:
assert MNIST.info.features['image'].shape == (28,28,1)
assert MNIST.info.features['label'].num_classes == 10
assert MNIST.info.splits['train'].num_examples == 60000

In [351]:
MNIST.download_and_prepare()

In [352]:
ds, info = tfds.load("mnist", split="train", with_info=True)
train_examples = tfds.as_dataframe(ds.take(10000), info)

In [353]:
train_images = np.asarray(train_examples['image'])
train_labels = np.asarray(train_examples['label'])

In [354]:
t2 = []
t4 = []

In [355]:
for i in range(len(train_images)):
    t2.append(train_images[i])
    t4.append(train_labels[i])

In [356]:
train_3 = np.asarray(t2)
train_y = np.asarray(t4)

In [357]:
train_y = to_categorical(train_y)

In [358]:
y_final = to_categorical(y_final)

In [359]:
rows = train.shape[1]
cols = train.shape[2]
stopper = EarlyStopping(monitor="val_loss", patience=10)

In [360]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [361]:
X_train = X_train.reshape(60000,28,28,1)
y_train = to_categorical(y_train)

In [362]:
new_train = np.concatenate((X_train, train, train_3), axis=0).astype("float32")
new_target_train = np.concatenate((y_train, y_final, train_y), axis=0)

In [363]:
def mod():
    model = Sequential()
    model.add(Conv2D(64, activation="relu", kernel_size=2, input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128, activation="relu", kernel_size=2))
    model.add(MaxPooling2D(pool_size=(2,2)))    
    model.add(Flatten())
    model.add(Dense(10, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [364]:
##### CALL LOAD MODEL FROM new_best_model #####
model = load_model("CNN_101121.hdf5")
model.fit(new_train, new_target_train, epochs=20, batch_size=64, verbose=1, validation_split=0.2, callbacks=[stopper])

Epoch 1/20
1400/1400 [==============================] - 37s 26ms/step - loss: 0.2178 - accuracy: 0.9983 - val_loss: 0.5881 - val_accuracy: 0.9977
Epoch 2/20
1400/1400 [==============================] - 39s 28ms/step - loss: 0.2065 - accuracy: 0.9986 - val_loss: 0.8303 - val_accuracy: 0.9971
Epoch 3/20
1400/1400 [==============================] - 37s 27ms/step - loss: 0.1991 - accuracy: 0.9984 - val_loss: 0.6271 - val_accuracy: 0.9974
Epoch 4/20
1400/1400 [==============================] - 39s 28ms/step - loss: 0.1868 - accuracy: 0.9987 - val_loss: 0.5405 - val_accuracy: 0.9983
Epoch 5/20
1400/1400 [==============================] - 39s 28ms/step - loss: 0.2001 - accuracy: 0.9985 - val_loss: 0.4623 - val_accuracy: 0.9979
Epoch 6/20
1400/1400 [==============================] - 38s 27ms/step - loss: 0.1681 - accuracy: 0.9987 - val_loss: 0.5875 - val_accuracy: 0.9980
Epoch 7/20
1400/1400 [==============================] - 37s 26ms/step - loss: 0.1705 - accuracy: 0.9988 - val_loss: 0.5236 -

In [365]:
X_test = test.values.reshape(28000,28,28,1)
predictions = model.predict(X_test)

In [366]:
predictions = pd.DataFrame(predictions).idxmax(axis=1)

In [367]:
submission = pd.DataFrame({"ImageId":np.arange(1,28001), "label":predictions})

In [368]:
submission.to_csv("mnist_101021_XI.csv", index=False)